In [ ]:
from google.colab import files

# Upload the kaggle.json file
files.upload()

In [ ]:
import os

# Make a directory for Kaggle API credentials
!mkdir -p ~/.kaggle

# Move kaggle.json to the directory
!mv kaggle.json ~/.kaggle/

# Set permissions (important for security)
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
#!kaggle datasets download -d nirmalsankalana/plant-diseases-training-dataset

In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

In [ ]:
!unzip new-plant-diseases-dataset.zip -d plant_disease_data

In [ ]:
#!unzip plant-diseases-training-dataset.zip -d dataset/

In [ ]:
import os
os.listdir()

In [ ]:
import zipfile

# Extract the new dataset ZIP file to "dataset/" folder
with zipfile.ZipFile("new-plant-diseases-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("dataset/")

In [ ]:
"""import zipfile

# Extract the ZIP file to "dataset/" folder
with zipfile.ZipFile("plant-diseases-training-dataset.zip", "r") as zip_ref:
    zip_ref.extractall("dataset/")

# List extracted files
import os
os.listdir("dataset/")

In [ ]:
os.listdir("dataset/")

In [ ]:
import os

print(os.listdir("dataset/"))  # Check what is inside the "dataset" folder

In [ ]:
print(os.listdir("dataset/New Plant Diseases Dataset(Augmented)/"))

In [ ]:
!pip install tensorflow

In [ ]:
import matplotlib.pyplot as plt
import os
import random
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the dataset path
train_dir = "/content/plant_disease_data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"

# Get class names
class_names = os.listdir(train_dir)

# Select a random class
random_class = random.choice(class_names)
class_path = os.path.join(train_dir, random_class)

# Get some images from the selected class
image_files = os.listdir(class_path)
random_images = random.sample(image_files, 5)  # Select 5 random images

# Plot images
fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for i, img_file in enumerate(random_images):
    img_path = os.path.join(class_path, img_file)
    img = load_img(img_path, target_size=(224, 224))  # Resize image
    axes[i].imshow(img)
    axes[i].axis("off")
    axes[i].set_title(random_class)  # Show class label

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
import numpy as np

In [ ]:
import os
base_dir = "/content/plant_disease_data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
print(os.listdir(base_dir))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation settings
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

 # Load a single image
sample_img_path = os.path.join(class_path, random_images[0])
sample_img = load_img(sample_img_path, target_size=(224, 224))
sample_array = img_to_array(sample_img)
sample_array = np.expand_dims(sample_array, axis=0)  # Add batch dimension

# Generate augmented images
fig, axes = plt.subplots(1, 5, figsize=(15, 5))
for i, batch in enumerate(datagen.flow(sample_array, batch_size=1)):
    aug_img = batch[0].astype("uint8")  # Convert to uint8 format for display
    axes[i].imshow(aug_img)
    axes[i].axis("off")
    if i == 4:  # Stop after 5 images
        break

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
import numpy as np

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

valid_datagen = ImageDataGenerator(rescale=1.0/255)

train_dir = "/content/plant_disease_data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_dir = "/content/plant_disease_data/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode="categorical"
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir, target_size=(224, 224), batch_size=32, class_mode="categorical"
)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

class_counts = Counter(valid_generator.classes)
plt.figure(figsize=(12,5))
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
plt.xticks(ticks=range(len(class_counts)), labels=valid_generator.class_indices.keys(), rotation=90)
plt.xlabel("Classes")
plt.ylabel("Number of Images")
plt.title("Class Distribution")
plt.show()

In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# 🔥 Unfreeze Last 5 Layers for Fine-Tuning
for layer in base_model.layers[:-5]:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)  # ✅ Better than Flatten
x = Dense(1024, activation="swish")(x)  # ✅ Swish > ReLU
x = BatchNormalization()(x)  # ✅ Normalize activations
x = Dropout(0.5)(x)  # ✅ Reduce overfitting

x = Dense(512, activation="swish")(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

output_layer = Dense(train_generator.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output_layer)

In [ ]:
model.compile(
    optimizer=AdamW(learning_rate=0.0001, weight_decay=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# 🔥 Reduce LR if No Improvement
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1, min_lr=1e-6)

# 🔥 Stop Training if No Improvement
early_stop = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)

# 🔥 Cyclical Learning Rate (CLR) for Fast Convergence
def clr_schedule(epoch):
    lr = 0.0001 * (0.85 ** epoch)  # Exponential Decay
    return lr

clr = LearningRateScheduler(clr_schedule)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,  # 🔥 10 Epochs Only
    callbacks=[reduce_lr, early_stop, clr],
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt

# Plot Accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Model Accuracy")
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Get true labels & predictions
y_true = valid_generator.classes
y_pred = model.predict(valid_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Confusion Matrix
conf_mat = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(12, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=valid_generator.class_indices.keys(), yticklabels=valid_generator.class_indices.keys())
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Classification Report
print(classification_report(y_true, y_pred_classes, target_names=valid_generator.class_indices.keys()))

In [ ]:
import random
from tensorflow.keras.preprocessing import image

# Get class labels
class_labels = list(valid_generator.class_indices.keys())

# Select random images
fig, axes = plt.subplots(3, 3, figsize=(10, 10))
for i in range(9):
    img_path, label = valid_generator.filepaths[i], class_labels[valid_generator.labels[i]]
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    pred = model.predict(img_array)
    predicted_label = class_labels[np.argmax(pred)]

     # Show Image
    ax = axes[i//3, i%3]
    ax.imshow(img)
    ax.set_title(f"Actual: {label}\nPredicted: {predicted_label}")
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
misclassified_idx = np.where(y_true != y_pred_classes)[0]

fig, axes = plt.subplots(3, 3, figsize=(10, 10))
for i, idx in enumerate(misclassified_idx[:9]):
    img_path = valid_generator.filepaths[idx]
    img = image.load_img(img_path, target_size=(224, 224))
    ax = axes[i//3, i%3]
    ax.imshow(img)
    ax.set_title(f"True: {class_labels[y_true[idx]]}\nPred: {class_labels[y_pred_classes[idx]]}")
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Save the Model
model.save("plant_disease_vgg16_optimized.h5")
print("✅ Model training complete & saved as 'plant_disease_vgg16_optimized.h5'")

In [ ]:
model.save("plant_disease_vgg16_optimized.keras")
print("✅ Model training complete & saved as 'plant_disease_vgg16_optimized.keras'")

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/content/plant_disease_vgg16_optimized.h5")
model = load_model("/content/plant_disease_vgg16_optimized.keras")

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/content/plant_disease_vgg16_optimized.h5")
img_path = "/content/istockphoto-172304886-612x612.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

pred = model.predict(img_array)
predicted_class = class_labels[np.argmax(pred)]
print("Predicted Class:", predicted_class)

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import json

# Load trained model
model = load_model("/content/plant_disease_vgg16_optimized.h5")

# Load class labels (Make sure this matches your training order)
class_labels = [
    "Apple Scab", "Apple Black Rot", "Apple Cedar Rust", "Apple Healthy",
    "Blueberry Healthy", "Cherry Powdery Mildew", "Cherry Healthy",
    "Corn Gray Leaf Spot", "Corn Common Rust", "Corn Northern Leaf Blight", "Corn Healthy",
    "Grape Black Rot", "Grape Esca (Black Measles)", "Grape Leaf Blight", "Grape Healthy",
    "Orange Citrus Greening", "Peach Bacterial Spot", "Peach Healthy",
    "Pepper Bell Bacterial Spot", "Pepper Bell Healthy",
    "Potato Early Blight", "Potato Late Blight", "Potato Healthy",
    "Raspberry Healthy", "Soybean Healthy",
    "Squash Powdery Mildew",
    "Strawberry Leaf Scorch", "Strawberry Healthy",
    "Tomato Bacterial Spot", "Tomato Early Blight", "Tomato Late Blight",
    "Tomato Leaf Mold", "Tomato Septoria Leaf Spot", "Tomato Spider Mites", "Tomato Target Spot",
    "Tomato Mosaic Virus", "Tomato Yellow Leaf Curl Virus", "Tomato Healthy"
]

# Function for preprocessing image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Predict function
def predict_disease(img_path):
    img_array = preprocess_image(img_path)
    pred = model.predict(img_array)
    predicted_class = class_labels[np.argmax(pred)]
    confidence = np.max(pred) * 100  # Confidence percentage
    return predicted_class, confidence

# Test on an image
img_path = "/content/frogeye-1.jpg"
predicted_class, confidence = predict_disease(img_path)

print(f"Predicted Disease: {predicted_class} ({confidence:.2f}%)")

In [ ]:
# Save the Model
model.save("plant_disease_vgg16_optimized.h5")
print("✅ Model training complete & saved as 'plant_disease_vgg16_optimized.h5'")

In [ ]:
model.save("plant_disease_vgg16_optimized.keras")
print("✅ Model training complete & saved as 'plant_disease_vgg16_optimized.keras'")